In [1]:
from kusanagi.ghost.algorithms.ExperienceDataset import ExperienceDataset
from functools import partial
import numpy as np

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 970 (0000:01:00.0)


In [2]:
import torch

global_dtype = torch.FloatTensor
#global_dtype = torch.cuda.FloatTensor

MATRIX_STRUCTURES = ['general', 'lower', 'upper', 'symmetric']
class Solve(torch.autograd.Function):
    def __init__(self, A_structure='general'):
        self.A_structure = A_structure
        
    def forward(self, A, b):
        if self.A_structure == 'lower':
            x = torch.trtrs(b, A, False)[0]
        elif self.A_structure == 'upper':
            x = torch.trtrs(b, A, True)[0]
        else:
            x = torch.gesv(b, A)[0]
        
        self.save_for_backward(A, b, x)

        return x
    
    def backward(self, output_grads):
        """
        Reverse-mode gradient updates for matrix solve operation c = A \ b.
        Symbolic expression for updates taken from [1]_.
        References
        ----------
        ..[1] M. B. Giles, "An extended collection of matrix derivative results
          for forward and reverse mode automatic differentiation",
          http://eprints.maths.ox.ac.uk/1079/
        """
        A, b, x = self.saved_tensors
        
        grad_x = output_grads
       
        # transpose solve
        if self.A_structure == 'lower':
            grad_b = torch.trtrs(b, A.t(), True)[0]
        elif self.A_structure == 'upper':
            grad_b = torch.trtrs(b, A.t(), False)[0]
        else:
            grad_b = torch.gesv(b, A.t())[0]
            
        # force outer product if vector second input
        grad_A = -grad_b[:,None].mm(x[None,:])\
                 if x.ndimension() == 1\
                 else -grad_b.mm(x.t())
                
        if self.A_structure == 'lower':
            grad_A = torch.tril(grad_A)
        elif self.A_structure == 'upper':
            grad_A = torch.triu(grad_A)
        
        return grad_A, grad_b
    
def solve_lower_triangular(A, b):
    return Solve(A_structure='lower')(A, b)

def solve_upper_triangular(A, b):
    return Solve(A_structure='upper')(A, b)

class Cholesky(torch.autograd.Function):
    def __init__(self, lower=True):
        self.lower = lower
        
    def forward(self, input):
        chol = torch.potrf(input, not self.lower)
        self.save_for_backward(input,chol)
        return chol
    
    def backward(self, output_grads):
        """
        Cholesky decomposition reverse-mode gradient update.
        Symbolic expression for reverse-mode Cholesky gradient taken from [0]_
        References
        ----------
        .. [0] I. Murray, "Differentiation of the Cholesky decomposition",
           http://arxiv.org/abs/1602.07527
        """

        x, chol_x = self.saved_tensors
        dz = output_grads
        
        # TODO deal with nans 
        
        if not self.lower:
            chol_x = chol_x.t()
            dz = dz.t()

        def tril_and_halve_diagonal(mtx):
            """Extracts lower triangle of square matrix and halves diagonal."""
            return torch.tril(mtx) - torch.diag(torch.diag(mtx)/2)
            
        def conjugate_solve_triangular(outer, inner):
            """Computes L^{-T} P L^{-1} for lower-triangular L."""
            return solve_upper_triangular(
                outer.t(), solve_upper_triangular(outer.t(), inner.t()).t())
        
     
        s = conjugate_solve_triangular(
            chol_x, tril_and_halve_diagonal(chol_x.t().mm(dz)))
        
        s2 = s + s.t()
        sdiag = torch.diag(torch.diag(s))
        
        if self.lower:
            grad = torch.tril(s2) - sdiag
        else:
            grad = torch.triu(s2) - sdiag
            
        return grad
    
def cholesky(input, lower=True):
    return Cholesky(lower)(input)

def maha(X,Y,M=None):
    if not M is None:
        XM = X.mm(M)
        YM = Y.mm(M)
        dist = (XM*X).sum(1).repeat(1, Y.size(0))\
             + (YM*Y).sum(1).t().repeat(X.size(0), 1)\
             - 2*XM.mm(Y.t())
    else:
        dist = (X**2).sum(1).repeat(1,Y.size(0))\
             + (Y**2).sum(1).t().repeat(X.size(0),1)\
             - 2*X.mm(Y.t())
    return dist

def SEard(loghyp, X, Y):
    if Y is None:
        Y = X
    n, D = X.size()
    dist = maha(X, Y, torch.diag(torch.exp(-2*loghyp[:D])))
    return torch.exp(2*loghyp[D].expand(dist.size()) - 0.5*dist )

def Noise(loghyp, X, Y=None):
    if Y is None:
        Y = X
    if X is Y:
        ones = torch.autograd.Variable(torch.ones(X.size(0)))
        K = ones*torch.exp(2*loghyp).expand(ones.size())
        return K.diag()
    else:
        return torch.zeros(X.size(0), X.size(0))
    
def Sum(loghyp_list,cov_list, X, Y=None):
    D = len(cov_list)
    K = sum([cov_list[i](loghyp_list[i], X, Y) for i in range(D)])
    return K

def SEard_params(X, Y):
    n, idims = X.size()
    odims = Y.size(1)
    
    loghyp = torch.autograd.Variable(torch.Tensor(odims,idims+1).type(global_dtype),
                                     requires_grad=True)
    loghyp.data[:, :idims] = 0.5*X.std(0).repeat(odims,1)
    loghyp.data[:, idims] = 0.5*Y.std(0)
    
    return loghyp

def Noise_params(X, Y):
    n, idims = X.size()
    odims = Y.size(1)
    
    loghyp = torch.autograd.Variable(torch.Tensor(odims,1), requires_grad=True)
    loghyp.data[:, 0] = 0.1*Y.std(0)
    
    return loghyp

def GP_loss(loghyps, covs, X, Y):
    if not type(covs) is list:
        covs= covs[covs]
        
    n, idims = X.size()
    odims = Y.size(1)
    
    loss = 0
    X_ = torch.autograd.Variable(X)
    loghyps_ = zip(*loghyps)
    
    for i in range(odims):
        K = Sum(loghyps_[i],covs, X_)
        L = cholesky(K)

In [3]:
exp = ExperienceDataset(filename='c:/Users/jcgamboahiguera/kusanagi/examples/learned_policies/cartpole/PILCO_SSGP_UI_Cartpole_RBFPolicy_sat_dataset')
X, Y = exp.get_dynmodel_dataset()
#convert to torch tensors
X, Y = torch.from_numpy(X).type(global_dtype),torch.from_numpy(Y).type(global_dtype)

[2017-06-16 17:11:33.952255] Experience > Loading state from c:/Users/jcgamboahiguera/kusanagi/examples/learned_policies/cartpole/PILCO_SSGP_UI_Cartpole_RBFPolicy_sat_dataset.zip


In [4]:
X_ = torch.autograd.Variable(X)
loghyps = SEard_params(X, Y), Noise_params(X,Y)
K = Sum(list(zip(*loghyps))[0],[SEard,Noise], X_, X_)

In [5]:
GP_loss(loghyps, [SEard, Noise], X, Y)

In [5]:
cholK = cholesky(K)
print(cholK.size())

torch.Size([800, 800])


In [6]:
S = torch.randn(800, 800)
S = S + S.t()
S = torch.autograd.Variable(S, requires_grad=True)
Y0 = torch.autograd.Variable(Y[:,0,None], requires_grad=True)

In [7]:
R = Solve('general')(S, Y0)
test_ = torch.eye(R.size(0)).view(R.size(0),1,R.size(0))
def vjp():

(Variable containing:
  0.0057
 -0.0556
  0.3713
 -0.2362
 -0.4194
  0.1312
 -0.2695
  0.3175
  0.4956
 -0.0331
 -0.3058
  0.1665
 -0.0740
 -0.2443
 -0.0427
  0.4696
 -0.1898
 -0.1400
 -0.2392
 -0.4488
 -0.3576
  0.3106
  0.2348
  0.6964
 -0.4666
  0.5703
 -0.1438
 -0.2647
 -0.0453
  0.1285
  0.2474
  0.0786
 -0.1980
  0.4264
  0.4965
  0.0320
 -0.7209
  0.1076
  0.1872
 -0.0288
 -0.4626
  0.1256
  0.2427
  0.1278
 -0.2071
  0.1853
  0.2104
 -0.4828
  0.2470
 -0.0944
 -0.6974
 -0.0365
 -0.3069
 -0.1974
 -0.2941
  0.2646
  0.1484
  0.2164
  0.3965
 -0.3464
  0.0524
  0.2672
  0.0288
 -0.1876
  0.0779
  0.1376
 -0.3034
  0.1568
  0.1350
 -0.1886
  0.1813
  0.3014
 -0.2159
  0.5240
  0.1754
  0.1902
 -0.2548
  0.3521
  0.0205
  0.0207
  0.4075
 -0.6382
  0.3337
  0.3772
 -0.0833
  0.0847
 -0.1878
 -0.0394
 -0.3982
 -0.1745
  0.0888
  0.2839
  0.2704
 -0.1230
  0.5504
  0.1915
 -0.3505
 -0.1432
 -0.0497
 -0.1323
 -0.1091
 -0.1909
  0.2567
  0.0568
  0.3376
  0.2368
  0.6110
 -0.0106
 -0.83

In [107]:
import theano
import numpy as np

# inti vars
s = theano.tensor.matrix('s')
y = theano.tensor.vector('y')
s_ = theano.tensor.slinalg.cholesky(s+1e2*theano.tensor.eye(s.shape[0]))
r = theano.tensor.slinalg.Solve()(s_, y)

# jacobian using scan
dr1 = theano.tensor.jacobian(r, s)

dummy_t = theano.tensor.ones((s.shape[0],))
dummy = theano.tensor.Lop(r, s, dummy_t)

from theano.scan_module.scan_utils import clone as t_clone
from theano.scan_module.scan_utils import map_variables as t_map

# jacobian using Lop and identity matrix
dr2, dr2_updts = theano.map(lambda t, r ,s: theano.tensor.Lop(r, s, t),
                            sequences=theano.tensor.eye(s.shape[0]),
                            non_sequences=[r, s])

dr3, dr3_updts = theano.map(lambda t, r ,s: theano.clone(dummy, replace={dummy_t: t}),
                            sequences=theano.tensor.eye(s.shape[0]),
                            non_sequences=[r, s])

# do it by replicating inputs
inp_list = [(s_.clone(), y.clone()) for i in range(r.shape[0])
r_list = [theano.tensor.slinalg.Solve()(si, yi) for si, yi in inp_list]

# compile funcs
f = theano.function(outputs=[r], inputs=[s, y])
df1 = theano.function(outputs=dr1, inputs=[s, y])
df2 = theano.function(outputs=dr2, inputs=[s, y], updates=dr2_updts)
df3 = theano.function(outputs=dr3, inputs=[s, y], updates=dr3_updts)

# test
#D = 200
#S = np.random.randn(D, D).astype(theano.config.floatX)
#S = S + S.T
#Y = np.ones(D).astype(theano.config.floatX)
R1 = df1(S.data[:100,:100].numpy(), Y.numpy()[:100,0])
R2 = df2(S.data[:100,:100].numpy(), Y.numpy()[:100,0])
R3 = df3(S.data[:100,:100].numpy(), Y.numpy()[:100,0])
#np.allclose(R1, R2.reshape(R1.shape), atol=1e-4, rtol=1e-4)

In [108]:
np.allclose(R2, R3.reshape(R1.shape), atol=1e-4, rtol=1e-4)

True

In [109]:
R1.shape, R2.shape

((100, 100, 100), (100, 100, 100))

In [120]:
%timeit df1(S.data[:100,:100].numpy(), Y.numpy()[:100,0])

157 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [121]:
%timeit df2(S.data[:100,:100].numpy(), Y.numpy()[:100,0])

152 ms ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [122]:
%timeit df3(S.data[:100,:100].numpy(), Y.numpy()[:100,0])

147 ms ± 8.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
theano.gof.graph.inputs([dummy],blockers=[t])

[s, y]